In [1]:
from word2gauss import embeddings

In [2]:
embeddings.GaussianEmbedding

word2gauss.embeddings.GaussianEmbedding

In [3]:
from word2gauss import GaussianEmbedding, iter_pairs
from vocab import Vocabulary

# v = Vocabulary()
# with open('./text8.txt','r', encoding='utf-8') as corpus:
#     v.create(corpus,[(10000000,5,350)])
#
# v.save('text8_voc.gz')  #save file

# load vocab
v = Vocabulary.load('F:/project_code/text8_voc.gz')



In [4]:
# embed =GaussianEmbedding(len(v),50,covariance_type='spherical', energy_type='KL')


In [ ]:
with open('F:/project_code/text8.txt','r', encoding='utf-8') as corpus:
    embed.train(iter_pairs(corpus,v),n_workers =8)

In [4]:
import numpy as np

In [5]:
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-VO.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])
        
        
# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val

In [7]:
dicA.pop(338)
dicB.pop(338)
val.pop(338)
dicA.pop(338)
dicB.pop(338)
val.pop(338)
dicA.pop(370)
dicB.pop(370)
val.pop(370)
dicA.pop(370)
dicB.pop(370)
val.pop(370)
dicA.pop(404)
dicB.pop(404)
val.pop(404)
dicA.pop(404)
dicB.pop(404)
val.pop(404)

0

In [47]:
# def KL_div(mui,muj, sigmai,sigmaj,K =50):
#     sigma_ratio = sigmai/sigmaj
#     det_fac = K*np.log(sigma_ratio)
#     trace_fac = K* sigma_ratio
    
#     mu_diff_sq = 0
#     for k in range(K):
#         mu_diff = mui[k]-muj[k]
#         mu_diff_sq += mu_diff* mu_diff
        
#     return 0.5*(trace_fac + mu_diff_sq/sigmaj - K - det_fac)

In [48]:
# def compute_KL(listA,listB,Gauss):
#     '''
#     imput two phrase or sentences lists
#     tokenize each pair and get a new gaussain by merging these distributions
#     compute KL pred for each pair

#     '''

#     n = len(listA)
#     KL_list = []
#     for i in range(n):
#         phraseA = listA[i].split()
#         phraseB = listB[i].split()
#         muA = 0.5*(Gauss.mu[v.word2id(phraseA[0].lower())] + Gauss.mu[v.word2id(phraseA[1].lower())])
#         muB = 0.5*(Gauss.mu[v.word2id(phraseB[0].lower())] + Gauss.mu[v.word2id(phraseB[1].lower())])
#         sigmaA = 0.5*(Gauss.sigma[v.word2id(phraseA[0].lower())] + Gauss.sigma[v.word2id(phraseA[1].lower())])
#         sigmaB = 0.5*(Gauss.sigma[v.word2id(phraseB[0].lower())] + Gauss.sigma[v.word2id(phraseB[1].lower())])
#         kl =KL_div(muA,muB,sigmaA,sigmaB)
#         KL_list.append(kl)

#     return KL_list


In [49]:
# pred = compute_KL(dicA, dicB,embed)
# from sklearn.metrics import roc_auc_score
# result = roc_auc_score(val, pred)
# print(result)

0.5040562466197944


In [8]:
embed2 =GaussianEmbedding(len(v),50,covariance_type='diagonal', energy_type='KL')


In [9]:
with open('F:/project_code/text8.txt','r', encoding='utf-8') as corpus:
    embed2.train(iter_pairs(corpus,v),n_workers =8)

In [10]:
def KL_div_dia(mui,muj, sigmai,sigmaj,K =50):
    trace_fac =0.0
    mu_diff_sq =0.0
    det_fac =1.0
    for k in range(K):
        sigma_ratio = sigmai[k]/sigmaj[k]
        trace_fac +=sigma_ratio
        mu_diff = mui[k]-muj[k]
        mu_diff_sq += mu_diff* mu_diff/sigmaj[k]
        
        det_fac *= sigma_ratio
        
        # boound def_fac
    if det_fac < 1.0e-8:
        det_fac = 1.0e-8
    elif det_fac > 1.0e8:
        det_fac = 1.0e8
        
    
#     sigma_ratio = sigmai/sigmaj
#     det_fac = K*np.log(sigma_ratio)
#     trace_fac = K* sigma_ratio
    
#     mu_diff_sq = 0
#     for k in range(K):
#         mu_diff = mui[k]-muj[k]
#         mu_diff_sq += mu_diff* mu_diff
        
    return 0.5*(trace_fac + mu_diff_sq - K - np.log(det_fac))

In [11]:
def compute_KL_dia(listA,listB,Gauss):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get a new gaussain by merging these distributions
    compute KL pred for each pair

    '''

    n = len(listA)
    R_list = []
    for i in range(n):
        phraseA = listA[i].split()
        phraseB = listB[i].split()
        muA = 0.5*(Gauss.mu[v.word2id(phraseA[0].lower())] + Gauss.mu[v.word2id(phraseA[1].lower())])
        muB = 0.5*(Gauss.mu[v.word2id(phraseB[0].lower())] + Gauss.mu[v.word2id(phraseB[1].lower())])
        sigmaA = 0.25*(Gauss.sigma[v.word2id(phraseA[0].lower())] + Gauss.sigma[v.word2id(phraseA[1].lower())])
        sigmaB = 0.25*(Gauss.sigma[v.word2id(phraseB[0].lower())] + Gauss.sigma[v.word2id(phraseB[1].lower())])
        kl =KL_div_dia(muA,muB,sigmaA,sigmaB)
        R_list.append(1/(1+kl))

    return R_list

In [12]:
pred2 = compute_KL_dia(dicA, dicB,embed2)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred2)
print(result)

0.4912925905895078


In [13]:
# get sv dataset
# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-SV.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])

# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store 

In [14]:
for i in range(len(dicA)):
    for j in dicA[i].split():
        try:
            t =embed2.mu[v.word2id(j)]
        except KeyError as e:
            print(i)

20
34
92
94
104
140
196
240
250


In [15]:
dicA.pop(20)
dicB.pop(20)
val.pop(20)
dicA.pop(20)
dicB.pop(20)
val.pop(20)

0

In [16]:
dicA.pop(32)
dicB.pop(32)
val.pop(32)
dicA.pop(32)
dicB.pop(32)
val.pop(32)

0

In [17]:
dicA.pop(96)
dicB.pop(96)
val.pop(96)
dicA.pop(96)
dicB.pop(96)
val.pop(96)
dicA.pop(130)
dicB.pop(130)
val.pop(130)
dicA.pop(130)
dicB.pop(130)
val.pop(130)

0

In [18]:
dicA.pop(88)
dicB.pop(88)
val.pop(88)
dicA.pop(88)
dicB.pop(88)
val.pop(88)
dicA.pop(88)
dicB.pop(88)
val.pop(88)
dicA.pop(88)
dicB.pop(88)
val.pop(88)

0

In [19]:
dicA.pop(184)
dicB.pop(184)
val.pop(184)
dicA.pop(184)
dicB.pop(184)
val.pop(184)
dicA.pop(226)
dicB.pop(226)
val.pop(226)
dicA.pop(226)
dicB.pop(226)
val.pop(226)
dicA.pop(234)
dicB.pop(234)
val.pop(234)
dicA.pop(234)
dicB.pop(234)
val.pop(234)

0

In [20]:
pred3 = compute_KL_dia(dicA, dicB,embed2)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred3)
print(result)

0.49162257495590833


In [21]:
# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-SVO.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])


# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val

In [22]:
for i in range(len(dicA)):
    for j in dicA[i].split():
        try:
            t =embed2.mu[v.word2id(j)]
        except KeyError as e:
            print(i)

26
38


In [23]:
dicA.pop(26)
dicB.pop(26)
val.pop(26)
dicA.pop(26)
dicB.pop(26)
val.pop(26)
dicA.pop(36)
dicB.pop(36)
val.pop(36)
dicA.pop(36)
dicB.pop(36)
val.pop(36)

0

In [24]:
def compute_KL_dia3(listA,listB,Gauss):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get a new gaussain by merging these distributions
    compute KL pred for each pair

    '''

    n = len(listA)
    R_list = []
    for i in range(n):
        phraseA = listA[i].split()
        phraseB = listB[i].split()
        muA_ = 0.5*(Gauss.mu[v.word2id(phraseA[0].lower())] + Gauss.mu[v.word2id(phraseA[1].lower())])
        muB_ = 0.5*(Gauss.mu[v.word2id(phraseB[0].lower())] + Gauss.mu[v.word2id(phraseB[1].lower())])
        sigmaA_ = 0.25*(Gauss.sigma[v.word2id(phraseA[0].lower())] + Gauss.sigma[v.word2id(phraseA[1].lower())])
        sigmaB_ = 0.25*(Gauss.sigma[v.word2id(phraseB[0].lower())] + Gauss.sigma[v.word2id(phraseB[1].lower())])
        muA =0.5*( muA_ + Gauss.mu[v.word2id(phraseA[2].lower())])
        muB = 0.5*( muB_ + Gauss.mu[v.word2id(phraseB[2].lower())])
        sigmaA = 0.25*(sigmaA_ + Gauss.sigma[v.word2id(phraseA[2].lower())])
        sigmaB = 0.25*(sigmaB_ + Gauss.sigma[v.word2id(phraseB[2].lower())])
        kl =KL_div_dia(muA,muB,sigmaA,sigmaB)
        R_list.append(1/(1+kl))

    return R_list

In [25]:
pred4 = compute_KL_dia3(dicA, dicB,embed2)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred4)
print(result)

0.49091695501730104
